In [1]:
import os

from series_extraction.excel_loader import ExcelLoader
from series_extraction.excel_validator import ExcelValidator
from series_extraction.excel_cleaner import ExcelCleaner
from series_extraction.table_finder import TableFinder
from series_extraction.series_extractor import SeriesExtractor
from series_extraction.excel_compatibility_checker import ExcelCompatibilityChecker
from series_extraction.series_iterator import SeriesIterator
from series_extraction.series_mapper import SeriesMapper

from ast_building.formula_parser import FormulaParser
from ast_building.series_implementer import SeriesImplementer

from ast_transformation.formula_generator import FormulaGenerator
from ast_transformation.formula_checker import FormulaChecker

from excel_utils import ExcelUtils

In [2]:
current_directory = os.getcwd()

parent_directory = os.path.abspath(os.path.join(current_directory, os.pardir))
data_directory = os.path.join(parent_directory, 'data')

project_name = 'test_excel_1'

excel_raw_file_path = os.path.join(data_directory, "excel_files_raw", f"{project_name}_raw.xlsx")
excel_reduced_filepath = os.path.join(data_directory, "excel_files_reduced", f"{project_name}_reduced.xlsx")

In [3]:
excel_raw= ExcelLoader.load_file(excel_raw_file_path)
excel_reduced = ExcelLoader.load_file(excel_reduced_filepath)

In [4]:
is_valid = ExcelValidator.validate_excel(excel_reduced)

In [5]:
if not is_valid:
    raise Exception("Excel file is not valid")

In [6]:
excel_reduced_clean = ExcelCleaner.clean_excel(excel_reduced)

In [7]:
extracted_tables, data = TableFinder.find_tables(excel_reduced_clean)

In [8]:
series_data = SeriesExtractor.extract_table_details(extracted_tables, data)

In [9]:
is_compatible = ExcelCompatibilityChecker.check_file(excel_raw, excel_reduced, extracted_tables)

In [10]:
if not is_compatible:
    raise Exception("Excel file is not compatible")

In [11]:
series_dict = SeriesExtractor.extract_series(extracted_tables=extracted_tables, data=data)

In [12]:
series_dict

{'Sheet1': [Series(series_id=SeriesId(sheet_name='Sheet1', series_header='horizontal_column_1', series_header_cell_row=12, series_header_cell_column=2), worksheet=Worksheet(sheet_name='Sheet1', workbook_file_path=None, worksheet=None), series_header='horizontal_column_1', formulas=['=B3', '=C3'], values=[1, 2], header_location=<HeaderLocation.LEFT: 'left'>, series_starting_cell=Cell(column=3, row=12, coordinate='C12', sheet_name=None, value=None, value_type=None), series_length=2, series_data_type=<SeriesDataType.INT: 'int'>),
  Series(series_id=SeriesId(sheet_name='Sheet1', series_header='horizontal_column_2', series_header_cell_row=13, series_header_cell_column=2), worksheet=Worksheet(sheet_name='Sheet1', workbook_file_path=None, worksheet=None), series_header='horizontal_column_2', formulas=['=B4', '=C4'], values=[3, 4], header_location=<HeaderLocation.LEFT: 'left'>, series_starting_cell=Cell(column=3, row=13, coordinate='C13', sheet_name=None, value=None, value_type=None), series_l

In [13]:
series_mapping = SeriesMapper.map_series(series_dict)

In [14]:
series_mapping

{Worksheet(sheet_name='Sheet1', workbook_file_path=None, worksheet=None): {Cell(column=3, row=12, coordinate=None, sheet_name=None, value=None, value_type=None): (0,
   Series(series_id=SeriesId(sheet_name='Sheet1', series_header='horizontal_column_1', series_header_cell_row=12, series_header_cell_column=2), worksheet=Worksheet(sheet_name='Sheet1', workbook_file_path=None, worksheet=None), series_header='horizontal_column_1', formulas=['=B3', '=C3'], values=[1, 2], header_location=<HeaderLocation.LEFT: 'left'>, series_starting_cell=Cell(column=3, row=12, coordinate='C12', sheet_name=None, value=None, value_type=None), series_length=2, series_data_type=<SeriesDataType.INT: 'int'>)),
  Cell(column=4, row=12, coordinate=None, sheet_name=None, value=None, value_type=None): (1,
   Series(series_id=SeriesId(sheet_name='Sheet1', series_header='horizontal_column_1', series_header_cell_row=12, series_header_cell_column=2), worksheet=Worksheet(sheet_name='Sheet1', workbook_file_path=None, worksh

In [15]:
series_iterator = SeriesIterator.iterate_series(series_dict)

In [16]:
series_list = [series for series in series_iterator if series.formulas != [None, None]]

In [17]:
import xlcalculator
import ast

from objects import Cell, HeaderLocation, CellRange, Column, CellRangeColumn

from excel_utils import ExcelUtils
from ast_transformation.formula_generator import SeriesIdLoader

class CellRangeImplementer:
    def __init__(self, sheet_name: str):
        self.sheet_name = sheet_name

    def merge_cell_ranges(self, cell_ranges):
        min_row = min(cell_range.start_cell.row for cell_range in cell_ranges)
        min_column = min(cell_range.start_cell.column for cell_range in cell_ranges)
        max_row = max(cell_range.end_cell.row for cell_range in cell_ranges)
        max_column = max(cell_range.end_cell.column for cell_range in cell_ranges)

        start_cell = Cell(column=min_column, row=min_row, coordinate = ExcelUtils.get_coordinate_from_column_and_row(min_column, min_row), sheet_name=self.sheet_name)
        end_cell = Cell(column=max_column, row=max_row, coordinate = ExcelUtils.get_coordinate_from_column_and_row(max_column, max_row), sheet_name=self.sheet_name)

        return CellRange(start_cell=start_cell, end_cell=end_cell)

    @staticmethod
    def create_cell_range_top_header(start_index, end_index, cell_row, cell_column):
        updated_start_cell_row = cell_row + start_index
        updated_end_cell_row = cell_row + end_index
        return CellRange(
            start_cell=Cell(row=updated_start_cell_row, column=cell_column, coordinate=ExcelUtils.get_coordinate_from_column_and_row(cell_column, updated_start_cell_row)),
            end_cell=Cell(row=updated_end_cell_row, column=cell_column + end_index, coordinate=ExcelUtils.get_coordinate_from_column_and_row(cell_column + end_index, updated_end_cell_row))
        )

    @staticmethod
    def create_cell_range_left_header(start_index, end_index, cell_row, cell_column, sheet_name):
        updated_start_cell_column = cell_column + start_index
        updated_end_cell_column = cell_column + end_index
        return CellRange(
            start_cell=Cell(row=cell_row, column=updated_start_cell_column, coordinate=ExcelUtils.get_coordinate_from_column_and_row(updated_start_cell_column, cell_row), sheet_name=sheet_name),
            end_cell=Cell(row=cell_row + end_index, column=updated_end_cell_column, coordinate=ExcelUtils.get_coordinate_from_column_and_row(updated_end_cell_column, cell_row + end_index), sheet_name=sheet_name)
        )
    
    @staticmethod
    def get_cell_range_from_series_tuple(series_tuple):
        series_ids_string, indexes = series_tuple
        series_start_index, series_end_index = indexes

        if series_start_index is None and series_end_index is None:
            return CellRangeImplementer.process_series_columns(series_ids_string)
        
        return CellRangeImplementer.process_series_cells(series_ids_string, series_start_index, series_end_index)

    @staticmethod
    def process_series_columns(series_ids_string):
        column_values = []
        for series_id_string in series_ids_string:
            column_value = CellRangeImplementer.get_column_from_series_id(series_id_string)
            column_values.append(column_value)

        sorted_column_values = sorted(column_values, key=lambda x: x.column_number)
        return CellRangeColumn(start_column=sorted_column_values[0], end_column=sorted_column_values[-1])

    @staticmethod
    def get_column_from_series_id(series_id_string):
        series_id = SeriesIdLoader.load_series_id_from_string(series_id_string)
        sheet_name = series_id.sheet_name
        series_list = series_dict.get(sheet_name)

        for series in series_list:
            if series.series_id == series_id:
                column_value = series.series_starting_cell.column
                return Column(column_number=column_value, column_letter=ExcelUtils.get_column_letter_from_number(column_value))

    @staticmethod
    def process_series_cells(series_ids_string, series_start_index, series_end_index):
        cell_ranges = []
        for series_id_string in series_ids_string:
            cell_range = CellRangeImplementer.get_cell_range_for_series_id(series_id_string, series_start_index, series_end_index)
            cell_ranges.append(cell_range)

        return CellRangeImplementer.merge_cell_ranges(cell_ranges)

    @staticmethod
    def get_cell_range_for_series_id(series_id_string, series_start_index, series_end_index):
        series_id = SeriesIdLoader.load_series_id_from_string(series_id_string)
        sheet_name = series_id.sheet_name
        series_list = series_dict.get(sheet_name)

        for series in series_list:
            if series.series_id == series_id:
                return CellRangeImplementer.create_cell_range(series, series_start_index, series_end_index)

    @staticmethod
    def create_cell_range(series, series_start_index, series_end_index):
        cell_value = series.series_starting_cell
        cell_row = cell_value.row
        cell_column = cell_value.column

        if series.header_location == HeaderLocation.TOP:
            return CellRangeImplementer.create_cell_range_top_header(series_start_index, series_end_index, cell_row, cell_column)
        elif series.header_location == HeaderLocation.LEFT:
            return CellRangeImplementer. create_cell_range_left_header(series_start_index, series_end_index, cell_row, cell_column)
        else:
            raise Exception("Header location is not valid")

    def update_ast(self, ast):
        if isinstance(ast, xlcalculator.ast_nodes.RangeNode):
            return self.replace_range_node(ast)
        elif isinstance(ast, xlcalculator.ast_nodes.FunctionNode):
            return self.replace_function_node(ast)
        elif isinstance(ast, xlcalculator.ast_nodes.OperatorNode):
            return self.replace_operator_node(ast)
        return ast

    def replace_range_node(self, node):

        series_tuple = ast.literal_eval(node.tvalue)
        cell_range = CellRangeImplementer.get_cell_range_from_series_tuple(series_tuple)

        return xlcalculator.ast_nodes.RangeNode(
            xlcalculator.tokenizer.f_token(
                tvalue=cell_range, ttype="operand", tsubtype="range"
            ))
        

    def replace_function_node(self, node):
        modified_args = [self.update_ast(arg) for arg in node.args]
        modified_function_node = xlcalculator.ast_nodes.FunctionNode(node.token)
        modified_function_node.args = modified_args
        return modified_function_node

    def replace_operator_node(self, node):
        modified_left = self.update_ast(node.left) if node.left else None
        modified_right = self.update_ast(node.right) if node.right else None
        modified_operator_node = xlcalculator.ast_nodes.OperatorNode(node.token)
        modified_operator_node.left = modified_left
        modified_operator_node.right = modified_right
        return modified_operator_node

In [18]:
for series in series_list:
    formula_1 = series.formulas[0]
    formula_1_ast = FormulaParser.parse_formula(formula_1)
    series_implementer = SeriesImplementer(series_mapping, sheet_name = series.worksheet.sheet_name)
    formula_1_ast_series = series_implementer.update_ast(formula_1_ast)

    formula_2 = series.formulas[1]
    formula_2_ast = FormulaParser.parse_formula(formula_2)
    formula_2_ast_series = series_implementer.update_ast(formula_2_ast)
    
    sheet_name = series.worksheet.sheet_name

    series_list = series_dict.get(sheet_name)

    ast_generator = FormulaGenerator.get_ast_generator(formula_1_ast_series, formula_2_ast_series, series_list)

    formulas_are_correct, formula_1_ast_new, formula_2_ast_new = FormulaChecker.check_formulas(ast_generator)

    cell_range_implementer = CellRangeImplementer(sheet_name=sheet_name)

    formula_1_ast_new_cell_ranges = cell_range_implementer.update_ast(formula_1_ast_new)
    formula_2_ast_new_cell_ranges = cell_range_implementer.update_ast(formula_2_ast_new)
    
    series.formulas = [f"={formula_1_ast_new_cell_ranges}", f"={formula_2_ast_new_cell_ranges}"]
    print(series.formulas)
    
    if not formulas_are_correct:
        print("formula_2_ast_series")
        print(formula_2_ast_series)
        print("formula_2_ast_new")
        print(formula_2_ast_new)
        raise Exception("Formulas are not correct")

['=None!B3:B3', '=None!C3:C3']
['=None!B4:C4', '=None!C4:D4']
['=(None!B3:B3) + (None!C3:C3)', '=(None!B4:C4) + (None!C4:D4)']
['=SUM(None!B3:D3)', '=SUM(None!B4:E4)']
['=(None!G14:G14) + (None!H14:H14)', '=(None!G15:H15) + (None!H15:I15)']
['=SUM(None!G14:I14)', '=SUM(None!G15:J15)']
['=None!B2:B2', '=None!B3:B3']
['=None!C2:C3', '=None!C3:C4']
['=(None!B2:B2) + (None!B3:B3)', '=(None!C2:C3) + (None!C3:C4)']
['=SUM(None!B2:C3)', '=SUM(None!B2:C3)']
['=SUM(None!C2:D3)', '=SUM(None!C2:D3)']
['=SUM(None!D2:E3)', '=SUM(None!D2:E3)']
['=(None!A2:A2) * (None!B2:B2)', '=(None!A3:B3) * (None!B3:C3)']


In [19]:

series_list

[Series(series_id=SeriesId(sheet_name='Sheet3', series_header='col_1', series_header_cell_row=1, series_header_cell_column=1), worksheet=Worksheet(sheet_name='Sheet3', workbook_file_path=None, worksheet=None), series_header='col_1', formulas=['=SUM(None!B2:C3)', '=SUM(None!B2:C3)'], values=[4, 4], header_location=<HeaderLocation.TOP: 'top'>, series_starting_cell=Cell(column=1, row=2, coordinate='A2', sheet_name=None, value=None, value_type=None), series_length=2, series_data_type=<SeriesDataType.INT: 'int'>),
 Series(series_id=SeriesId(sheet_name='Sheet3', series_header='col_2', series_header_cell_row=1, series_header_cell_column=2), worksheet=Worksheet(sheet_name='Sheet3', workbook_file_path=None, worksheet=None), series_header='col_2', formulas=['=SUM(None!C2:D3)', '=SUM(None!C2:D3)'], values=[6, 6], header_location=<HeaderLocation.TOP: 'top'>, series_starting_cell=Cell(column=2, row=2, coordinate='B2', sheet_name=None, value=None, value_type=None), series_length=2, series_data_type=